In [1]:
import os
import sys
import torch
import time

base_directory = "../"
sys.path.insert(0, base_directory)

output_directory = "./output/noise-tests/sd2-notebook"
os.makedirs(output_directory, exist_ok=True)

from stable_diffusion2.latent_diffusion import LatentDiffusion
from stable_diffusion2.stable_diffusion import StableDiffusion
from stable_diffusion2.utils.model import *
from stable_diffusion2.utils.utils import SectionManager as section
from stable_diffusion2.utils.utils import *
from stable_diffusion2.model.clip.clip_embedder import CLIPTextEmbedder



from stable_diffusion2.model.unet.unet import UNetModel

from pathlib import Path
from os.path import join

device = get_device()

INFO: Using CUDA device: NVIDIA GeForce RTX 3080 Ti


In [2]:
def logistic_distribution(loc, scale):
    base_distribution = torch.distributions.Uniform(0, 1)
    transforms = [torch.distributions.transforms.SigmoidTransform().inv, torch.distributions.transforms.AffineTransform(loc=loc, scale=scale)]
    logistic = torch.distributions.TransformedDistribution(base_distribution, transforms)
    return logistic
noise_fn = lambda shape, device = device: logistic_distribution(loc=0.0, scale=0.5).sample(shape).to(device)

In [3]:
stable_diffusion_model = StableDiffusion(device=device)
get_memory_status();

INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Total: 12287 MiB
Free: 11084 MiB
Used: 1203 MiB


In [4]:
stable_diffusion_model.model.load_submodel_tree()
get_memory_status();

Starting section: load submodel tree...
Finished section: load submodel tree in 2.73 seconds

Total: 12287 MiB
Free: 6932 MiB
Used: 5355 MiB


In [5]:
stable_diffusion_model.unload_model()
torch.cuda.empty_cache()
get_memory_status();

Total: 12287 MiB
Free: 11084 MiB
Used: 1203 MiB


In [6]:
ddim_eta = 0.00000
stable_diffusion_model = StableDiffusion(device = device, ddim_eta=ddim_eta)
stable_diffusion_model.model.load_submodel_tree()
get_memory_status();

INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Starting section: load submodel tree...
Finished section: load submodel tree in 2.48 seconds

Total: 12287 MiB
Free: 6932 MiB
Used: 5355 MiB


In [7]:
temperature = 1.0
imgs = stable_diffusion_model.generate_images(
    prompt = 'A woman with flowers in her hair in a courtyard, in the style of Frank Frazetta',
    seed = 2982,
    noise_fn = noise_fn,
    temperature = temperature,
    )

save_images(imgs, join(output_directory, f'test_sample.png'))

Starting section: getting text cond...
Finished section: getting text cond in 0.90 seconds

Starting section: sampling...
Finished section: sampling in 5.49 seconds



In [8]:
stable_diffusion_model.unload_model()
del imgs
torch.cuda.empty_cache()
get_memory_status();

Total: 12287 MiB
Free: 9986 MiB
Used: 2301 MiB


In [9]:
img = load_img("./input/test_img.jpg").to(device)
get_memory_status();

Total: 12287 MiB
Free: 9986 MiB
Used: 2301 MiB


In [10]:
stable_diffusion_model.model.load_autoencoder()
get_memory_status();

Total: 12287 MiB
Free: 9986 MiB
Used: 2301 MiB


In [11]:
stable_diffusion_model.model.autoencoder.load_encoder()
get_memory_status();

Total: 12287 MiB
Free: 9846 MiB
Used: 2441 MiB


In [12]:
encoded_img = stable_diffusion_model.encode(img)
get_memory_status();

Total: 12287 MiB
Free: 9136 MiB
Used: 3151 MiB


In [13]:
encoded_img.shape

torch.Size([1, 4, 64, 64])

In [14]:
save_images(encoded_img[:, :3,:,:], join(output_directory, f'test_encoding_1.png'))
save_images(encoded_img[:, 1:,:,:], join(output_directory, f'test_encoding_2.png'))

In [28]:
torch.save(encoded_img, join(output_directory, f'encoded_img_temp{temperature}_eta{ddim_eta:.3f}.pt'))

In [29]:
del encoded_img
torch.cuda.empty_cache()
print(get_memory_status())

Total: 12287 MiB
Free: 9760 MiB
Used: 2527 MiB
(tensor(9760), tensor(12287))


In [30]:
encoded_img = torch.load(join(output_directory, f'encoded_img_temp{temperature}_eta{ddim_eta:.3f}.pt'))
torch.cuda.empty_cache()
get_memory_status()
encoded_img.shape

Total: 12287 MiB
Free: 9760 MiB
Used: 2527 MiB


torch.Size([1, 4, 64, 64])

In [31]:
torch.cuda.empty_cache()
get_memory_status();

Total: 12287 MiB
Free: 9760 MiB
Used: 2527 MiB


In [32]:
stable_diffusion_model.model.autoencoder.load_decoder()
torch.cuda.empty_cache()
get_memory_status();

Total: 12287 MiB
Free: 9726 MiB
Used: 2561 MiB


In [33]:
decoded_img = stable_diffusion_model.decode(encoded_img)
save_images(decoded_img, join(output_directory, f'decoded_img_temp{temperature}_eta{ddim_eta:.3f}.png'))
torch.cuda.empty_cache()
print(get_memory_status())

Total: 12287 MiB
Free: 9746 MiB
Used: 2541 MiB
(tensor(9746), tensor(12287))
